In [25]:
!pip install transformers


In [26]:
!pip install tensorboardx

In [27]:
!pip install simpletransformers


In [28]:
!pip install scikit-learn

In [29]:
import pandas as pd


#train_df = pd.read_csv('data/train.csv', header=None)
df = pd.read_csv('./data/newsdata.csv', sep='\t', names=['ID','TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

In [30]:
df = df[['TITLE','CATEGORY']]
#train_df['text'] = train_df.iloc[:, 1] + " " + train_df.iloc[:, 2]

In [31]:
encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))

#train_df = train_df.drop(train_df.columns[[1, 2]], axis=1)

In [32]:
#train_df.columns = ['label', 'text']
df

,TITLE,CATEGORY,ENCODE_CAT
0,"Fed official says weak data caused by weather,...",b,0
1,Fed's Charles Plosser sees high bar for change...,b,0
2,US open: Stocks fall after Fed official hints ...,b,0
3,"Fed risks falling 'behind the curve', Charles ...",b,0
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b,0
...,...,...,...
1094,Titanfall has 34GB of Uncompressed Audio Files...,t,1
1095,Titanfall Xbox One resolution hits 792p in fin...,t,1
1096,Titanfall live-action content to be produced b...,t,1
1097,inFamous: Second Son vs. Titanfall: Launch Eve...,t,1


In [33]:
#train_df = train_df[['text', 'label']]
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)

In [34]:
#train_df['text'] = train_df['text'].apply(lambda x: x.replace('\\', ' '))
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)

In [35]:
#train_df['label'] = train_df['label'].apply(lambda x:x-1)
print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (1099, 3)
TRAIN Dataset: (879, 3)
TEST Dataset: (220, 3)


In [36]:
#train_df
train_dataset = train_dataset[['TITLE','ENCODE_CAT']]

In [37]:
# eval_df = pd.read_csv('data/test.csv', header=None)
# eval_df['text'] = eval_df.iloc[:, 1] + " " + eval_df.iloc[:, 2]
# eval_df = eval_df.drop(eval_df.columns[[1, 2]], axis=1)
# eval_df.columns = ['label', 'text']
# eval_df = eval_df[['text', 'label']]
# eval_df['text'] = eval_df['text'].apply(lambda x: x.replace('\\', ' '))
# eval_df['label'] = eval_df['label'].apply(lambda x:x-1)
test_dataset = test_dataset[['TITLE','ENCODE_CAT']]

In [38]:
from simpletransformers.classification import ClassificationModel


In [46]:
model = ClassificationModel('distilbert', 'distilbert-base-uncased', args={'overwrite_output_dir':True}, use_cuda=False, num_labels=4)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [47]:
model.train_model(train_dataset)

/usr/local/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
  0%|          | 0/879 [00:00<?, ?it/s]2022-11-16 15:06:47.455788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 15:06:47.456350: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch 1 of 1: 100%|██████████| 1/1 [10:18<00:00, 618.81s/it]


(110, 0.5262718360532413)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_dataset, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
  0%|          | 0/220 [00:00<?, ?it/s]2022-11-16 03:31:21.577773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running Evaluation: 100%|██████████| 28/28 [00:25<00:00,  1.08it/s]


In [ ]:
result

{'mcc': 0.0,
 'f1': 0.8954545454545455,
 'acc': 0.8954545454545455,
 'eval_loss': 0.30470552747803076}

In [ ]:
model_outputs

array([[ 2.99142218e+00,  3.41591299e-01, -2.35672784e+00,
        -2.14168859e+00],
       [ 2.85016870e+00,  6.41761005e-01, -2.31629276e+00,
        -2.22711658e+00],
       [ 3.08885622e+00,  6.93988204e-02, -2.19317818e+00,
        -2.03435731e+00],
       [ 2.38172030e+00,  1.04416490e+00, -2.27251220e+00,
        -2.16002512e+00],
       [ 2.60644531e+00,  9.56777573e-01, -2.38703537e+00,
        -2.22655368e+00],
       [ 2.32081795e+00,  1.06330013e+00, -2.27044010e+00,
        -2.14715695e+00],
       [ 2.47914362e+00,  1.00309849e+00, -2.33419728e+00,
        -2.19582129e+00],
       [ 3.01877141e+00,  1.23470731e-01, -2.23166037e+00,
        -2.07933712e+00],
       [ 3.02130938e+00,  4.36623782e-01, -2.36105680e+00,
        -2.18987036e+00],
       [ 2.91967249e+00,  1.91520303e-01, -2.13052559e+00,
        -1.94655395e+00],
       [ 2.27261281e+00,  1.08818901e+00, -2.24918914e+00,
        -2.14997554e+00],
       [ 3.06454134e+00,  6.94136471e-02, -2.25878501e+00,
      

In [ ]:
wrong_predictions

[{'guid': 197, 'text_a': 'Microsoft Xbox One Titanfall Another Non-1080p Title', 'text_b': None, 'label': 1},
 {'guid': 198, 'text_a': 'Titanfall review in progress', 'text_b': None, 'label': 1},
 {'guid': 199, 'text_a': "Microsoft's Xbox Is on Its Last Life", 'text_b': None, 'label': 1},
 {'guid': 200, 'text_a': 'Respawn considering patching 1080p support into Xbox One Titanfall', 'text_b': None, 'label': 1},
 {'guid': 201, 'text_a': 'Titanfall Xbox One Resolution: Titanfall on Xbox One Resolution is 792p, "likely  ...', 'text_b': None, 'label': 1},
 {'guid': 202, 'text_a': 'Titanfall requires day one patch update, early Xbox One players report', 'text_b': None, 'label': 1},
 {'guid': 203, 'text_a': 'Review: Titanfall breathes new life into the boring FPS genre', 'text_b': None, 'label': 1},
 {'guid': 204, 'text_a': 'Titanfall review round-up: A heavyweight', 'text_b': None, 'label': 1},
 {'guid': 205, 'text_a': "Titanfall's 'Life is Better With a Titan' Trailer Gets an Extended Cut",

In [ ]:
predictions, raw_outputs = model.predict(['Bitcoin enthusiasm weathers chaotic week'])

  0%|          | 0/1 [00:00<?, ?it/s]2022-11-16 03:32:10.713270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00,  7.25it/s]


In [ ]:
predictions

array([0])